### Loading libraries

In [9]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np

### Web scrapping

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url, header=0)
table = tables[0]
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
table.shape

(180, 3)

In [12]:
table.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [13]:
table.Neighbourhood.value_counts()

Not assigned                                                                            77
Downsview                                                                                4
Don Mills                                                                                2
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale                   1
Bayview Village                                                                          1
The Annex, North Midtown, Yorkville                                                      1
Agincourt                                                                                1
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park                        1
University of Toronto, Harbord                                                           1
Humberlea, Emery                                                                         1
Runnymede, Swansea                                                                       1

In [14]:
table.Borough.replace("Not assigned", np.nan, inplace = True)
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop the rows where borough has Nan.


In [15]:
table.dropna(axis=0, inplace=True)
table = table.reset_index()
table = table.drop(['index'], axis=1)
table.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Group Neighbourhoods with the same Postcode


In [16]:
table = table.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
table = table.reset_index()
table.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Replace the 'Not aasigned' to 'Queen's Park' in Neighborhood


In [17]:
table = table.replace({'Not assigned' : "Queen's Park"}) 
table.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
table.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

###  Get shape of Table

In [27]:
table.shape

(103, 3)

### Load the coordinates from the csv file 

In [19]:
import pandas as pd

In [20]:
coordinates = pd.read_csv("C:\\Users\\pranit\\Desktop\\CpastoneProject\\Capstone-Project\\Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# rename the column "PostalCode"
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### merge two table on the column "PostalCode"

In [28]:
toronto_df_new = table.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df


C:\Users\pranit\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Borough,Latitude,Longitude,Neighborhood,Neighbourhood,PostalCode
0,Downtown Toronto,43.657952,-79.387383,NaN,Central Bay Street,M5G
1,North York,43.803762,-79.363452,NaN,Hillcrest Village,M2H
2,East York,43.706397,-79.309937,NaN,"Parkview Hill, Woodbine Gardens",M4B
3,Scarborough,43.744734,-79.239476,NaN,Scarborough Village,M1J
4,East York,43.709060,-79.363452,NaN,Leaside,M4G
5,East Toronto,43.659526,-79.340923,NaN,Studio District,M4M
6,Scarborough,43.750072,-79.295849,NaN,"Wexford, Maryvale",M1R
7,Etobicoke,43.739416,-79.588437,NaN,"South Steeles, Silverstone, Humbergate, Jamest...",M9V
8,North York,43.756303,-79.565963,NaN,Humber Summit,M9L
9,Downtown Toronto,43.628947,-79.394420,NaN,"CN Tower, King and Spadina, Railway Lands, Har...",M5V
